In [20]:
import pandas as pd
import json
from utils.data_cleaning import text_cleaning, text_cleaning_lighter, timestamp_to_year, extract_numeric_rating

In [21]:
books = pd.read_json('./data/book_best_001_050.jl', lines=True)
books.head(3)
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11403 entries, 0 to 11402
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   url              11403 non-null  object 
 1   title            11384 non-null  object 
 2   titleComplete    11384 non-null  object 
 3   description      11369 non-null  object 
 4   imageUrl         11368 non-null  object 
 5   genres           11185 non-null  object 
 6   asin             9838 non-null   object 
 7   isbn             9073 non-null   object 
 8   isbn13           9243 non-null   float64
 9   publisher        10972 non-null  object 
 10  author           11384 non-null  object 
 11  publishDate      11332 non-null  float64
 12  characters       6411 non-null   object 
 13  places           5362 non-null   object 
 14  ratingHistogram  11384 non-null  object 
 15  ratingsCount     11368 non-null  float64
 16  reviewsCount     11347 non-null  float64
 17  numPages    

In [22]:
interactives = pd.read_json('./data/interactive_test.json')
interactives.head(3)

,book_url,title,user,user_rate,user_review
0,https://www.goodreads.com/book/show/136251.Har...,Harry Potter and the Deathly Hallows,992038-tara,Rating 1 out of 5,"Ok, before I start a few warnings. This will c..."
1,https://www.goodreads.com/book/show/136251.Har...,Harry Potter and the Deathly Hallows,167451-suzanne,Rating 5 out of 5,"""I'm going to keep going until I succeed — or ..."
2,https://www.goodreads.com/book/show/136251.Har...,Harry Potter and the Deathly Hallows,8937622-kassidy,Rating 5 out of 5,It's hard for me to believe that I finished th...


# Data cleanning
## **Books**: information of 999xxxxxx books

**Drop some columns**

In [23]:
books.drop(['titleComplete', 'imageUrl', 'asin', 'isbn', 'isbn13', 'publisher', 'series', 'characters', 'places', 'awards'], axis=1, inplace = True)
# books.info()
books.head(3)

,url,title,description,genres,author,publishDate,ratingHistogram,ratingsCount,reviewsCount,numPages,language,avgRating
0,https://www.goodreads.com/book/show/2165.The_O...,The Old Man and the Sea,Librarian's note: An alternate cover edition c...,"[Adventure, Fiction, Literary Fiction, America...",[Ernest Hemingway],8.204832e+11,"[47620, 91381, 253168, 362501, 355777]",1110447.0,37865.0,96.0,English,NaN
1,https://www.goodreads.com/book/show/295.Treasu...,Treasure Island,"""For sheer storytelling delight and pure adven...","[Pirates, Adventure, Fiction, Classics, Fantas...","[N.C. Wyeth, Robert Louis Stevenson]",1.000537e+12,"[8528, 28655, 128405, 176772, 136475]",478835.0,15923.0,352.0,English,NaN
2,https://www.goodreads.com/book/show/7244.The_P...,The Poisonwood Bible,The Poisonwood Bible is a story told by the wi...,"[Historical, Fiction, Literary Fiction, Classi...",[Barbara Kingsolver],1.117523e+12,"[23947, 39130, 112637, 227128, 329801]",732643.0,26807.0,546.0,English,NaN


**Drop books with the same *url* or uncrawlable books**

In [24]:
books.drop_duplicates(subset='url', keep='first', inplace=True)
books.dropna(subset=['title'], inplace=True)
# title duplicates because of reprints, versions, publising companies
books.drop_duplicates(subset='title', keep='first', inplace=True)

**Books: *title*, *author*, *description***  
- convert *title*, *author* to str type
- fill *description*  missing values with default value (= *title*)
- translate *description* to English, remove meaningless words, convert to lower case, remove punctuations, special charecters, emojis and multiple spaces

In [25]:
books['title'] = books['title'].astype(str)
books['author'] = books['author'].apply(lambda x: ', '.join(x) if x else '')
books['description'].fillna(books['title'], inplace=True)
# cái này chạy hơi lâu, có thể xảy ra lỗi timeout, cmt để chạy test các function khác
books['description'] = books['description'].apply(text_cleaning_lighter)

**Books: *genres*, *language***  
- convert *genres*, *language* to categorical variables
- fill *genres*, *language* missing values with default value

In [26]:
books['genres'].fillna('', inplace=True)
# check and handle non-list-like data
books['genres'] = books['genres'].apply(lambda x: x if isinstance(x, list) else [])
books['genres'] = books['genres'].apply(lambda x: pd.Categorical(x))
# default language should be the most popular language 
books['language'].fillna('English', inplace=True)
books['language'] = books['language'].astype('category')

**Book: *publishDate***
- convert Unix timestamp *publishDate* to *publishYear*
- *avgPublishDate* of a book = mean of *publishDate* of books that had same *author* | If any and its *publishDate* is not missing  
  Else  *avgPublishDate* of a book = mean of *publishDate* of all books
- drop *publishDate* col

In [27]:
books['publishDate'] = books['publishDate'].apply(timestamp_to_year)
books = books.rename(columns={'publishDate': 'publishYear'})


author_avg_date = books.groupby('author')['publishYear'].mean()
overall_avg_date = books['publishYear'].mean()
books['publishYear'] = books.apply(
    lambda row: author_avg_date[row['author']] if pd.isnull(row['publishYear']) and row['author'] in author_avg_date.index else row['publishYear'],
    axis=1
)
books['publishYear'].fillna(overall_avg_date, inplace=True)
books['publishYear'] = books['publishYear'].astype(int)

**Book: *reviewsCount*, *ratingsCount*, *avgRating*, *ratingHistogram***
- fill *ratingsCount*, *avgRating* missing values with value calculated based on the *ratingHistogram*
- fill *reviewsCount* missing values with 0

In [28]:
books['reviewsCount'].fillna(0, inplace=True)

def calculate_avgRating(ratingHistogram):
  ratingsCount = sum(ratingHistogram)
  if ratingsCount:
    return sum(i * (idx + 1) for idx, i in enumerate(ratingHistogram)) / ratingsCount
  else:
    return 0

books['ratingsCount'] = books.apply(lambda row: sum(row['ratingHistogram']) if pd.isnull(row['ratingsCount']) else row['ratingsCount'], axis=1)
books['avgRating'] = books.apply(lambda row: calculate_avgRating(row['ratingHistogram']) if pd.isnull(row['avgRating']) else row['avgRating'], axis=1)
books['reviewsCount'] = books['reviewsCount'].astype(int)
books['ratingsCount'] = books['ratingsCount'].astype(int)


In [29]:
avgNumPages = books['numPages'].mean()
books['numPages'].fillna(avgNumPages, inplace=True)
books['numPages'] = books['numPages'].astype(int)

books.isnull().sum()
# books[books['avgRating'].isnull()]
# books.loc[books['author'] == 'Gordon R. Dickson']

url                0
title              0
description        0
genres             0
author             0
publishYear        0
ratingHistogram    0
ratingsCount       0
reviewsCount       0
numPages           0
language           0
avgRating          0
dtype: int64

In [30]:
books.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9636 entries, 0 to 10985
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   url              9636 non-null   object  
 1   title            9636 non-null   object  
 2   description      9636 non-null   object  
 3   genres           9636 non-null   object  
 4   author           9636 non-null   object  
 5   publishYear      9636 non-null   int32   
 6   ratingHistogram  9636 non-null   object  
 7   ratingsCount     9636 non-null   int32   
 8   reviewsCount     9636 non-null   int32   
 9   numPages         9636 non-null   int32   
 10  language         9636 non-null   category
 11  avgRating        9636 non-null   float64 
dtypes: category(1), float64(1), int32(4), object(6)
memory usage: 763.5+ KB


In [31]:
books.head(3)

,url,title,description,genres,author,publishYear,ratingHistogram,ratingsCount,reviewsCount,numPages,language,avgRating
0,https://www.goodreads.com/book/show/2165.The_O...,The Old Man and the Sea,librarian s note an alternate cover edition ca...,"['Adventure', 'Fiction', 'Literary Fiction', '...",Ernest Hemingway,1996,"[47620, 91381, 253168, 362501, 355777]",1110447,37865,96,English,3.799168
1,https://www.goodreads.com/book/show/295.Treasu...,Treasure Island,for sheer storytelling delight and pure advent...,"['Pirates', 'Adventure', 'Fiction', 'Classics'...","N.C. Wyeth, Robert Louis Stevenson",2001,"[8528, 28655, 128405, 176772, 136475]",478835,15923,352,English,3.843737
2,https://www.goodreads.com/book/show/7244.The_P...,The Poisonwood Bible,the poisonwood bible is a story told by the wi...,"['Historical', 'Fiction', 'Literary Fiction', ...",Barbara Kingsolver,2005,"[23947, 39130, 112637, 227128, 329801]",732643,26807,546,English,4.091536


## **Interactives**: user reviews and user ratings for the above books

In [32]:
interactives.head(3)

,book_url,title,user,user_rate,user_review
0,https://www.goodreads.com/book/show/136251.Har...,Harry Potter and the Deathly Hallows,992038-tara,Rating 1 out of 5,"Ok, before I start a few warnings. This will c..."
1,https://www.goodreads.com/book/show/136251.Har...,Harry Potter and the Deathly Hallows,167451-suzanne,Rating 5 out of 5,"""I'm going to keep going until I succeed — or ..."
2,https://www.goodreads.com/book/show/136251.Har...,Harry Potter and the Deathly Hallows,8937622-kassidy,Rating 5 out of 5,It's hard for me to believe that I finished th...


In [33]:
interactives.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   book_url     1470 non-null   object
 1   title        1470 non-null   object
 2   user         1430 non-null   object
 3   user_rate    1434 non-null   object
 4   user_review  1470 non-null   object
dtypes: object(5)
memory usage: 57.6+ KB


In [34]:
interactives.isnull().sum()
# interactives[interactives['user'].isnull()]

book_url        0
title           0
user           40
user_rate      36
user_review     0
dtype: int64

**Drop rows where both *user_rate* and *user_review* are missing**

In [35]:
interactives.dropna(subset=['user_rate', 'user_review'], inplace=True)

Interactives: *user_rate*
- convert to numeric

In [36]:
interactives['user_rate'] = interactives['user_rate'].apply(extract_numeric_rating) 

Interactives: *user_review*
- translate *user_review* to English, remove meaningless words, convert to lower case, remove punctuations, special charecters, emojis and multiple spaces
- Due to limitations of the web version of google translate, googletrans API does not guarantee that the library would work properly at all times  
  *text_cleaning_lighter* has removed the translation function and the meaningless words removing function  
  It’s probably, Google has banned your client IP address  
  TODO: use [Google’s official translate API](https://cloud.google.com/translate/docs)

In [37]:
# interactives['user_review'].fillna(interactives['title'], inplace=True)
interactives['user_review'] = interactives['user_review'].astype(str)

interactives['user_review'] = interactives['user_review'].apply(text_cleaning_lighter)

In [39]:
interactives

,book_url,title,user,user_rate,user_review
0,https://www.goodreads.com/book/show/136251.Har...,Harry Potter and the Deathly Hallows,992038-tara,1,ok before i start a few warnings this will con...
1,https://www.goodreads.com/book/show/136251.Har...,Harry Potter and the Deathly Hallows,167451-suzanne,5,i m going to keep going until i succeed or die...
2,https://www.goodreads.com/book/show/136251.Har...,Harry Potter and the Deathly Hallows,8937622-kassidy,5,it s hard for me to believe that i finished th...
3,https://www.goodreads.com/book/show/136251.Har...,Harry Potter and the Deathly Hallows,22106879-jayson,5,a 86 extraordinarynotes it ends too expository...
4,https://www.goodreads.com/book/show/136251.Har...,Harry Potter and the Deathly Hallows,30728719-lily,5,i can t believe its over i ve finally read the...
...,...,...,...,...,...
1464,https://www.goodreads.com/book/show/1656001.Th...,The Host,26259115-kevin-irish-reader,4,finally read this for the first time and it wa...
1466,https://www.goodreads.com/book/show/1656001.Th...,The Host,72729-karin,5,wanderer has just arrived on earth and been im...
1467,https://www.goodreads.com/book/show/1656001.Th...,The Host,36102568-chloe,5,i have to say i was not expecting this book to...
1468,https://www.goodreads.com/book/show/1656001.Th...,The Host,17945431-hristina,5,first read may 26th through june 8th 2013secon...


# Data Understanding by Visualization